📌 SECTION 1 — Import + Load

Cell 1 — Import

In [2]:
import pandas as pd

Cell 2 — Load Excel with 12 Sheets Name

In [4]:
file_path = '../datasets/Support_Report_2025.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)
all_sheets.keys()

dict_keys(['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12'])

SECTION 2 — Combine All Months

Cell 3 — รวมข้อมูลทั้งปี

In [9]:
df = pd.concat(all_sheets.values(), ignore_index=True)
df.head()

,ปัญหาที่ได้รับ,ผู้แจ้งปัญหา,วิธีการแก้ไข,Asset tag,การเปลี่ยน OA,优先事项,Start 日期,End 日期,JOB,Location,Dept,Status 状态,ผู้ดูแล,year_month
0,ตรวจสอบเครื่องมีปัญหา,พนักงานใหม่,ติดตั้งโปรแกรมสำเร็จ,JTTH/CO/98437,ไม่มี,ด่วน,2025-01-06 12:48:00,2025-01-06 20:48,Computer,HQ,HQ,Failed,@jirapat,2025-01
1,ลบโปรแกรม wps,CS Team,ลบโปรแกรมเรียบร้อย,JTTH/CO/97254,ไม่มี,ด่วน,2025-01-15 09:37:00,2025-01-15 17:37,CCTV,FN,CS,Pending,@wuttichai,2025-01
2,สร้าง printer code ให้ทีม,พนักงานใหม่,ติดตั้งโปรแกรมสำเร็จ,JTTH/CO/93182,ไม่มี,ปกติ,2025-01-13 15:18:00,2025-01-13 22:18,Software,CS,MKT,Pending,@suphansa,2025-01
3,แก้ network ช้า,พนักงานใหม่,แก้ network แล้ว,JTTH/CO/90502,มี,ด่วน,2025-01-09 17:53:00,2025-01-10 01:53,Software,HQ,Other,Failed,@suphansa,2025-01
4,set com ให้พนักงานใหม่,ฝ่ายบัญชี,reset account แล้ว,JTTH/CO/92374,มี,ปกติ,2025-01-19 15:11:00,2025-01-19 19:11,Software,HQ,FN,Success,@phitchaya,2025-01


SECTION 3 — Prepare Date Columns

เพราะเราจะใช้วันที่ทำรายวัน/รายเดือน/รายปี

Cell 4 — แปลงวันที่ให้เป็น datetime

In [6]:
df["Start 日期"] = pd.to_datetime(df["Start 日期"])

Cell 5 — เพิ่มคอลัมน์ year, month, year-month

In [8]:
df["year"] = df["Start 日期"].dt.year
df["month"] = df["Start 日期"].dt.month
df["year_month"] = df["Start 日期"].dt.strftime("%Y-%m")

📌 SECTION 4 — รายงานพื้นฐาน

Cell 6 — สรุปจำนวนงานรายเดือน

In [7]:
monthly_count = df.groupby("year_month").size()
monthly_count


year_month
2025-01    150
2025-02    135
2025-03    126
2025-04    133
2025-05    133
2025-06    126
2025-07    144
2025-08    135
2025-09    139
2025-10    138
2025-11    142
2025-12    134
dtype: int64

Cell 7 — สรุปประเภทงาน (JOB) รายเดือน

In [11]:
job_monthly = df.groupby(['year_month', 'JOB']).size()
job_monthly

year_month  JOB     
2025-01     Access      25
            CCTV        25
            Computer    22
            Network     20
            Printer     27
                        ..
2025-12     CCTV        23
            Computer    20
            Network     19
            Printer     17
            Software    23
Length: 72, dtype: int64

Cell 8 — สรุปจำนวนงานตามสถานะ

In [12]:
status_count = df.groupby('Status 状态').size()
status_count

Status 状态
Failed     537
Pending    571
Success    527
dtype: int64

Cell 9 — สรุปงานตามแผนก (Dept)

In [13]:
dept_count = df.groupby('Dept').size()
dept_count

Dept
CS       336
FN       332
HQ       299
MKT      363
Other    305
dtype: int64

📌 SECTION 5 — รายงานแบบ Pivot (เหมือน Excel แต่เทพกว่า)

Cell 10 — Pivot: เดือน × JOB

In [14]:
pivot_job = pd.pivot_table(
    df,
    index='year_month',
    columns='JOB',
    values='ปัญหาที่ได้รับ',
    aggfunc='count',
    fill_value=0
)
pivot_job

JOB,Access,CCTV,Computer,Network,Printer,Software
year_month,,,,,,
2025-01,25,25,22,20,27,31
2025-02,18,24,24,29,17,23
2025-03,21,18,16,23,22,26
2025-04,14,23,22,23,23,28
2025-05,18,19,23,25,20,28
2025-06,19,21,23,20,21,22
2025-07,28,24,22,24,25,21
2025-08,22,20,28,21,18,26
2025-09,23,26,24,24,21,21


Cell 11 — Pivot: เดือน × Status

In [15]:
pivot_status = pd.pivot_table(
    df,
    index='year_month',
    columns='Status 状态',
    values='ปัญหาที่ได้รับ',
    aggfunc='count',
    fill_value=00
)
pivot_status

Status 状态,Failed,Pending,Success
year_month,,,
2025-01,57,49,44
2025-02,42,51,42
2025-03,43,39,44
2025-04,42,45,46
2025-05,46,40,47
2025-06,41,46,39
2025-07,44,53,47
2025-08,45,43,47
2025-09,41,53,45


📌 SECTION 6 — Export รายงานออกเป็น Excel พร้อมหลายชีต

Cell 12 — Export

In [17]:
with pd.ExcelWriter('Summary_2025.xlsx') as writer:
    monthly_count.to_excel(writer, sheet_name='Monthly_Count')
    status_count.to_excel(writer, sheet_name='Status_Count')
    dept_count.to_excel(writer, sheet_name='Dept_Count')
    pivot_job.to_excel(writer, sheet_name='Pivot_Job')
    pivot_status.to_excel(writer, sheet_name='Pivot_Status')

'สร้างไฟล์ Summary_2025.xlsx แล้ว'

'สร้างไฟล์ Summary_2025.xlsx แล้ว'